xlm_roberta_large_xnli modell <br>
submission to kaggle: load submissions and return it, kaggle needs a submission code which returns a submission.csv file

In [1]:
!pip install sentencepiece
!pip3 install torch torchvision torchaudio
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import os
import gc

import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# set a seed value
torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, accuracy_score

import transformers
from transformers import AutoTokenizer, XLMRobertaForSequenceClassification
from transformers import AdamW

from datasets import load_dataset

import warnings
warnings.filterwarnings("ignore")


print(torch.__version__)

1.13.0+cu116


In [3]:
MODEL_TYPE = 'joeddav/xlm-roberta-large-xnli'


L_RATE = 1e-5
MAX_LEN = 256

NUM_EPOCHS = 5
BATCH_SIZE = 32
NUM_CORES = os.cpu_count()

NUM_CORES

12

In [4]:
device = torch.device('cuda:0')
device

device(type='cuda', index=0)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
df_train=pd.read_csv("drive/MyDrive/NLP_Project/data/train.csv")
df_val=pd.read_csv("drive/MyDrive/NLP_Project/data/valid.csv")
df_test=pd.read_csv("drive/MyDrive/NLP_Project/data/test.csv")

print(df_train.shape)

(10908, 6)


### Load mnli data

In [7]:
mnli = load_dataset('glue', 'mnli')
df_mlni = pd.DataFrame.from_dict(mnli["train"])
df_mlni.drop(columns=['idx'], inplace=True)
df_mlni.head()

  0%|          | 0/5 [00:00<?, ?it/s]

,premise,hypothesis,label
0,Conceptually cream skimming has two basic dime...,Product and geography are what make cream skim...,1
1,you know during the season and i guess at at y...,You lose the things to the following level if ...,0
2,One of our number will carry out your instruct...,A member of my team will execute your orders w...,0
3,How do you know? All this is their information...,This information belongs to them.,0
4,yeah i tell you what though if you go price so...,The tennis shoes have a range of prices.,1


In [8]:
mnli

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

In [9]:
df_mnli = pd.DataFrame.from_dict(mnli["validation_matched"])
df_mnli.drop(columns=['idx'], inplace=True)
df_mnli.shape

(9815, 3)

In [10]:
df_mnli['label'] = df_mnli['label'].replace([0, 2], [2, 0])       # because 0, 2 are changed in training of xlm_roberta_large_xnli
df_val['label'] = df_val['label'].replace([0, 2], [2, 0])                     # because 0, 2 are changed in training of xlm_roberta_large_xnli

### Load Tokenizer

In [11]:
print('Loading Roberta tokenizer...')
tokenizer = AutoTokenizer.from_pretrained(MODEL_TYPE)

Loading Roberta tokenizer...


In [12]:
df_val = df_val.reset_index(drop=True)
df_train_mnli = df_mnli.reset_index(drop=True)

In [13]:
class CompDataset(Dataset):

    def __init__(self, df):
        self.df_data = df



    def __getitem__(self, index):

        # get the sentence from the dataframe
        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']

        # Process the sentence
        # ---------------------

        encoded_dict = tokenizer.encode_plus(
                    sentence1, sentence2,           # Sentences to encode.
                    add_special_tokens = True,      # Add the special tokens.
                    max_length = MAX_LEN,
                    truncation=True,           # Pad & truncate all sentences.
                    pad_to_max_length = True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',          # Return pytorch tensors.
               )
        
        # These are torch tensors.
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        
        # Convert the target to a torch tensor
        target = torch.tensor(self.df_data.loc[index, 'label'])

        sample = (padded_token_list, att_mask, target)


        return sample


    def __len__(self):
        return len(self.df_data)
    

class TestDataset(Dataset):

    def __init__(self, df):
        self.df_data = df



    def __getitem__(self, index):

        # get the sentence from the dataframe
        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']

        # Process the sentence
        # ---------------------

        encoded_dict = tokenizer.encode_plus(
                    sentence1, sentence2,           # Sentence to encode.
                    add_special_tokens = True,      # Add the special tokens.
                    max_length = MAX_LEN,           # Pad & truncate all sentences.
                    truncation=True,           # Pad & truncate all sentences.
                    pad_to_max_length = True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',          # Return pytorch tensors.
               )
        
        # These are torch tensors.
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        
               

        sample = (padded_token_list, att_mask)


        return sample


    def __len__(self):
        return len(self.df_data)

In [14]:
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained(
    MODEL_TYPE, 
    num_labels = 3, # The number of output labels. 2 for binary classification.
)

# Send the model to the device.
model.to(device)

You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
        

In [15]:
# Define the optimizer
optimizer = AdamW(model.parameters(),
              lr = L_RATE, 
              eps = 1e-8 
            )

In [16]:
# Create the dataloaders.

train_data = CompDataset(df_mnli)
val_data = CompDataset(df_val)
test_data = TestDataset(df_test)

train_dataloader = torch.utils.data.DataLoader(train_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                       num_workers=NUM_CORES)

val_dataloader = torch.utils.data.DataLoader(val_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                       num_workers=NUM_CORES)

test_dataloader = torch.utils.data.DataLoader(test_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=False,
                                       num_workers=NUM_CORES)



print(len(train_dataloader))
print(len(val_dataloader))
print(len(test_dataloader))

307
38
163


In [17]:
# Set the seed.
seed_val = 400

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []


# For each epoch...
for epoch in range(0, NUM_EPOCHS):
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))
    

    stacked_val_labels = []
    targets_list = []


# ========================================
    #               Training
    # ========================================
    
    print('Training on ...')
    print(model.device)
    
    # put the model into train mode
    model.train()
    
    # This turns gradient calculations on and off.
    torch.set_grad_enabled(True)


    # Reset the total loss for this epoch.
    total_train_loss = 0

    for i, batch in enumerate(train_dataloader):
        
        train_status = 'Batch ' + str(i) + ' of ' + str(len(train_dataloader))
        
        print(train_status, end='\r')


        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()

        outputs = model(b_input_ids, 
                    attention_mask=b_input_mask,
                    labels=b_labels)
        
        # Get the loss from the outputs tuple: (loss, logits)
        loss = outputs[0]
        
        # Convert the loss from a torch tensor to a number.
        # Calculate the total loss.
        total_train_loss = total_train_loss + loss.item()
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        
        
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        
        
        # Use the optimizer to update the weights.
        
        # Optimizer for GPU
        optimizer.step() 
        


    
    print('Train loss:' ,total_train_loss)



    # ========================================
    #               Validation
    # ========================================
    
    print('\nValidation...')

    # Put the model in evaluation mode.
    model.eval()

    # Turn off the gradient calculations.
    # This tells the model not to compute or store gradients.
    # This step saves memory and speeds up validation.
    torch.set_grad_enabled(False)
    
    
    # Reset the total loss for this epoch.
    total_val_loss = 0
    

    for j, batch in enumerate(val_dataloader):
            
            val_status = 'Batch ' + str(j) + ' of ' + str(len(val_dataloader))
            
            print(val_status, end='\r')

            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)     


            outputs = model(b_input_ids, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
            
            # Get the loss from the outputs tuple: (loss, logits)
            loss = outputs[0]
            
            # Convert the loss from a torch tensor to a number.
            # Calculate the total loss.
            total_val_loss = total_val_loss + loss.item()
            

            # Get the preds
            preds = outputs[1]


            # Move preds to the CPU
            val_preds = preds.detach().cpu().numpy()
            
            # Move the labels to the cpu
            targets_np = b_labels.to('cpu').numpy()

            # Append the labels to a numpy list
            targets_list.extend(targets_np)

            if j == 0:  # first batch
                stacked_val_preds = val_preds
            else:
                stacked_val_preds = np.vstack((stacked_val_preds, val_preds))

        
        # Calculate the validation accuracy
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)
        
    val_acc = accuracy_score(y_true, y_pred)
        
        
    print('Val loss:' ,total_val_loss)
    print('Val acc: ', val_acc)


  # Use the garbage collector to save memory.
    gc.collect()
# Save the Model
torch.save(model.state_dict(), 'drive/MyDrive/NLP_Project/data/xlm_roberta_large_xnli_mlni_mnli_val_matched.pt')

    


======== Epoch 1 / 5 ========
Training on ...
cuda:0
Train loss: 107.74848488718271

Validation...
Val loss: 3.709232073277235
Val acc:  0.9678217821782178

======== Epoch 2 / 5 ========
Training on ...
cuda:0
Train loss: 55.394875794649124

Validation...
Val loss: 2.1274984730407596
Val acc:  0.9801980198019802

======== Epoch 3 / 5 ========
Training on ...
cuda:0
Train loss: 31.020631619845517

Validation...
Val loss: 2.3481050117406994
Val acc:  0.9843234323432343

======== Epoch 4 / 5 ========
Training on ...
cuda:0
Train loss: 23.558650817663874

Validation...
Val loss: 2.401188115880359
Val acc:  0.9876237623762376

======== Epoch 5 / 5 ========
Training on ...
cuda:0
Train loss: 19.330399783531902

Validation...
Val loss: 2.9874719649378676
Val acc:  0.985973597359736


In [18]:
for j, batch in enumerate(test_dataloader):
        
        inference_status = 'Batch ' + str(j+1) + ' of ' + str(len(test_dataloader))
        
        print(inference_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)


        outputs = model(b_input_ids, 
                attention_mask=b_input_mask)
        
        
        # Get the preds
        preds = outputs[0]


        # Move preds to the CPU
        preds = preds.detach().cpu().numpy()
        
        # Move the labels to the cpu
        targets_np = b_labels.to('cpu').numpy()

        # Append the labels to a numpy list
        targets_list.extend(targets_np)
        
        # Stack the predictions.

        if j == 0:  # first batch
            stacked_preds = preds

        else:
            stacked_preds = np.vstack((stacked_preds, preds))

In [19]:
# Take the argmax. This returns the column index of the max value in each row.

preds = np.argmax(stacked_preds, axis=1)

preds

array([0, 1, 2, ..., 2, 2, 0])

In [20]:
path = 'drive/MyDrive/NLP_Project/data/sample_submission.csv'

df_sample = pd.read_csv(path)

print(df_sample.shape)

df_sample.head()

(5195, 2)


,id,prediction
0,c6d58c3f69,1
1,cefcc82292,1
2,e98005252c,1
3,58518c10ba,1
4,c32b0d16df,1


In [21]:
df_sample['prediction'] = preds
df_sample['prediction'] = df_sample['prediction'].replace([0, 2], [2, 0])

df_sample.head()

,id,prediction
0,c6d58c3f69,2
1,cefcc82292,1
2,e98005252c,0
3,58518c10ba,1
4,c32b0d16df,1


In [22]:
# Create a submission csv file
# Note that for this competition the submission file must be named submission.csv.
# Therefore, it won't be possible to submit this csv file for leaderboard scoring.
seed = 400
df_sample.to_csv(f'drive/MyDrive/NLP_Project/data/xlm roberta_large_mnli_submission_seed{seed}.csv', index=False)